## 딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회

**※주의** : 반드시 본 파일을 이용하여 제출을 수행해야 하며, 파일의 이름은 `task.ipynb`로 유지되어야 합니다.

* #### 추론 실행 환경
    * `python 3.9` 환경
    * `CUDA 10.2`, `CUDA 11.8`, `CUDA 12.6`를 지원합니다.
    * 각 CUDA 환경에 미리 설치돼있는 torch 버전은 다음 표를 참고하세요.

<table>
  <thead>
    <tr>
      <th align="center">Python</th>
      <th align="center">CUDA</th>
      <th align="center">torch</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="center" style="vertical-align: middle;">3.8</td>
      <td align="center">10.2</td>
      <td align="center">1.6.0</td>
    </tr>
    <tr>
      <td align="center" style="vertical-align: middle;">3.9</td>
      <td align="center">11.8</td>
      <td align="center">1.8.0</td>
    </tr>
    <tr>
      <td align="center">3.10</td>
      <td align="center">12.6</td>
      <td align="center">2.7.1</td>
    </tr>
  </tbody>
</table>

* #### CUDA 버전 관련 안내사항  
  - 이번 경진대회는 3개의 CUDA 버전을 지원합니다.  
  - 참가자는 자신의 모델의 라이브러리 의존성에 맞는 CUDA 환경을 선택하여 모델을 제출하면 됩니다.   
  - 각 CUDA 환경에는 기본적으로 torch가 설치되어 있으나, 참가자는 제출하는 CUDA 버전과 호환되는 torch, 필요한 버전의 라이브러리를 `!pip install` 하여 사용하여도 무관합니다.

* #### `task.ipynb` 작성 규칙
코드는 크게 3가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하세요.   
1. **제출용 aifactory 라이브러리 및 추가 필요 라이브러리 설치**
    - 채점 및 제출을 위한 aifactory 라이브러리를 설치하는 셀입니다. 이 부분은 수정하지 않고 그대로 실행합니다.
    - 그 외로, 모델 추론에 필요한 라이브러리를 직접 설치합니다.
2. **추론용 코드 작성**
    - 모델 로드, 데이터 전처리, 예측 등 실제 추론을 수행하는 모든 코드를 이 영역에 작성합니다.
3. **aif.submit() 함수를 호출하여 최종 결과를 제출**
    - **마이 페이지-활동히스토리**에서 발급받은 key 값을 함수의 인자로 정확히 입력해야 합니다.
    - **※주의** : 제출하고자 하는 CUDA 환경에 맞는 key를 입력하여야 합니다.

<table>
  <thead>
    <tr>
      <th align="left">Competition 이름</th>
      <th align="center">CUDA</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회</td>
      <td align="center">11.8</td>
    </tr>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회 CUDA 12.6</td>
      <td align="center">12.6</td>
    </tr>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회 CUDA 10.2</td>
      <td align="center">10.2</td>
    </tr>
  </tbody>
</table>

------

#### 1. 제출용 aifactory 라이브러리 설치
※ 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [16]:
!pip install -U aifactory

* 자신의 모델 추론 실행에 필요한 추가 라이브러리 설치

In [ ]:
# 대회 서버 환경: Python 3.10 + CUDA 12.6 + torch 2.7.1 (기본 설치)
# 커스텀 ViT 모델에 필요한 라이브러리 설치
!pip install pandas --no-cache-dir --quiet
!pip install timm==0.9.12 --no-cache-dir --quiet
!pip install einops --no-cache-dir --quiet
!pip install torchvision --no-cache-dir --quiet
!pip install numpy --no-cache-dir --quiet
!pip install opencv-python-headless --no-cache-dir --quiet
!pip install Pillow --no-cache-dir --quiet
!pip install dlib --no-cache-dir --quiet
!pip install tqdm --no-cache-dir --quiet
!pip install mediapipe --no-cache-dir --quiet

# 리소스 모니터링 (대회 환경 디버깅용)
!pip install psutil --no-cache-dir --quiet

# LLaVA 추론에 필요한 라이브러리 설치
!pip install transformers --no-cache-dir --quiet
!pip install accelerate --no-cache-dir --quiet
!pip install safetensors --no-cache-dir --quiet
!pip install sentencepiece --no-cache-dir --quiet
!pip install protobuf --no-cache-dir --quiet

# LLaVA 라이브러리 설치 (GitHub에서 직접 설치)
!pip install git+https://github.com/haotian-liu/LLaVA.git --no-cache-dir --quiet

-----

#### 2. 추론용 코드 작성

##### 추론 환경의 기본 경로 구조

- 평가 데이터셋 경로: `./data/`
   - 채점에 사용될 테스트 데이터셋은 `./data/` 디렉토리 안에 포함되어 있습니다.
   - 해당 디렉토리에는 이미지(JPG, PNG)와 동영상(MP4) 파일이 별도의 하위 폴더 없이 혼합되어 있습니다.
```bash
/aif/
└── data/
    ├── {이미지 데이터1}.jpg
    ├── {이미지 데이터2}.png
    ├── {동영상 데이터1}.mp4
    ├── {이미지 데이터3}.png
    ├── {동영상 데이터2}.mp4
    ...
```

- 모델 및 자원 경로: 예시 : `./model/`
   - 추론 스크립트가 실행되는 위치를 기준으로, 제출된 모델 관련 파일들이 위치해야하 하는 상대 경로입니다.
   - 학습된 모델 가중치(.pt, .ckpt, .pth 등)

* 제출 파일은 `submission.csv`로 저장돼야 합니다.
  * submission.csv는 *filename*과 *label* 컬럼으로 구성돼야 합니다.
  * filename은 추론한 파일의 이름(확장자 포함), label은 추론 결과입니다. (real:0, fake:1)
  * filename은 *string*, label은 *int* 자료형이어야 합니다.
  * 추론하는 데이터의 순서는 무작위로 섞여도 상관 없습니다.

<table>
  <thead>
    <tr>
      <th align="center">filename</th>
      <th align="center">label</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="center">{이미지 데이터1}.jpg</td>
      <td align="center">0</td>
    </tr>
    <tr>
      <td align="center">{동영상 데이터1}.mp4</td>
      <td align="center">1</td>
    </tr>
    <tr>
      <td colspan="2" align="center">...</td>
    </tr>
  </tbody>
</table>

**※ 주의 사항**

* argparse 사용시 `args, _ = parser.parse_known_args()`로 인자를 지정하세요.   
   - `args = parser.parse_args()`는 jupyter에서 오류가 발생합니다.
* return 할 결과물과 양식에 유의하세요.

In [ ]:
import os
import sys
from PIL import Image
import cv2
import dlib
from pathlib import Path
import numpy as np
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torchvision import transforms
import time
import warnings
warnings.filterwarnings('ignore')

# stdout/stderr 에러 억제
import sys
import os

# Mediapipe 및 TensorFlow Lite 로그 억제 (import 전에 설정)
os.environ['GLOG_minloglevel'] = '3'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# ============================================================
# 리소스 모니터링 유틸리티 (대회 환경 리소스 측정용)
# ============================================================
try:
    import psutil
    PSUTIL_AVAILABLE = True
except ImportError:
    PSUTIL_AVAILABLE = False
    print("⚠️  psutil not available - CPU/RAM monitoring disabled")

class ResourceMonitor:
    """대회 환경의 리소스 사용량을 측정하는 클래스"""
    
    def __init__(self, device='cuda'):
        self.device = device
        self.is_cuda = device == 'cuda' and torch.cuda.is_available()
        self.process = psutil.Process() if PSUTIL_AVAILABLE else None
        
        # 통계 저장
        self.stats = {
            'gpu_memory_peak': 0,
            'gpu_memory_current': 0,
            'cpu_memory_peak': 0,
            'cpu_memory_current': 0,
            'inference_times': [],
            'preprocessing_times': [],
            'total_files_processed': 0
        }
        
    def get_model_size(self, model):
        """모델의 파라미터 수와 메모리 크기 계산"""
        param_count = sum(p.numel() for p in model.parameters())
        param_size = sum(p.numel() * p.element_size() for p in model.parameters())
        buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())
        total_size_mb = (param_size + buffer_size) / (1024 ** 2)
        
        return {
            'total_params': param_count,
            'trainable_params': sum(p.numel() for p in model.parameters() if p.requires_grad),
            'size_mb': total_size_mb
        }
    
    def get_gpu_memory(self):
        """GPU 메모리 사용량 조회 (MB 단위)"""
        if not self.is_cuda:
            return {'allocated': 0, 'reserved': 0, 'max_allocated': 0}
        
        return {
            'allocated': torch.cuda.memory_allocated() / (1024 ** 2),  # MB
            'reserved': torch.cuda.memory_reserved() / (1024 ** 2),
            'max_allocated': torch.cuda.max_memory_allocated() / (1024 ** 2)
        }
    
    def get_cpu_memory(self):
        """CPU 메모리 사용량 조회 (MB 단위)"""
        if not PSUTIL_AVAILABLE:
            return {'rss': 0, 'vms': 0, 'percent': 0}
        
        mem_info = self.process.memory_info()
        return {
            'rss': mem_info.rss / (1024 ** 2),  # MB (실제 물리 메모리)
            'vms': mem_info.vms / (1024 ** 2),  # MB (가상 메모리)
            'percent': self.process.memory_percent()
        }
    
    def get_gpu_utilization(self):
        """GPU 사용률 조회 (nvidia-smi 필요)"""
        if not self.is_cuda:
            return None
        try:
            import subprocess
            result = subprocess.run(
                ['nvidia-smi', '--query-gpu=utilization.gpu,temperature.gpu', '--format=csv,noheader,nounits'],
                capture_output=True, text=True, timeout=1
            )
            if result.returncode == 0:
                util, temp = result.stdout.strip().split(',')
                return {'utilization': float(util), 'temperature': float(temp)}
        except:
            pass
        return None
    
    def update_stats(self):
        """현재 리소스 사용량 업데이트"""
        # GPU 메모리
        gpu_mem = self.get_gpu_memory()
        self.stats['gpu_memory_current'] = gpu_mem['allocated']
        self.stats['gpu_memory_peak'] = max(self.stats['gpu_memory_peak'], gpu_mem['max_allocated'])
        
        # CPU 메모리
        cpu_mem = self.get_cpu_memory()
        self.stats['cpu_memory_current'] = cpu_mem['rss']
        self.stats['cpu_memory_peak'] = max(self.stats['cpu_memory_peak'], cpu_mem['rss'])
    
    def log_inference_time(self, elapsed_time):
        """추론 시간 기록"""
        self.stats['inference_times'].append(elapsed_time)
        self.stats['total_files_processed'] += 1
    
    def log_preprocessing_time(self, elapsed_time):
        """전처리 시간 기록"""
        self.stats['preprocessing_times'].append(elapsed_time)
    
    def print_current_status(self, prefix=""):
        """현재 리소스 상태 출력"""
        gpu_mem = self.get_gpu_memory()
        cpu_mem = self.get_cpu_memory()
        gpu_util = self.get_gpu_utilization()
        
        print(f"\n{prefix}[Resource Status]")
        print(f"  GPU Memory: {gpu_mem['allocated']:.1f} MB (Peak: {gpu_mem['max_allocated']:.1f} MB)")
        print(f"  CPU Memory: {cpu_mem['rss']:.1f} MB ({cpu_mem['percent']:.1f}%)")
        if gpu_util:
            print(f"  GPU Util: {gpu_util['utilization']:.1f}% | Temp: {gpu_util['temperature']:.1f}°C")
    
    def print_summary(self):
        """최종 리소스 사용량 요약 출력"""
        print("\n" + "="*60)
        print("RESOURCE USAGE SUMMARY")
        print("="*60)
        
        # 모델 정보는 별도로 출력되므로 생략
        
        # GPU 메모리
        print(f"\n[GPU Memory (VRAM)]")
        print(f"  Peak Usage: {self.stats['gpu_memory_peak']:.1f} MB")
        print(f"  Current: {self.stats['gpu_memory_current']:.1f} MB")
        
        # CPU 메모리
        print(f"\n[CPU Memory (RAM)]")
        print(f"  Peak Usage: {self.stats['cpu_memory_peak']:.1f} MB")
        print(f"  Current: {self.stats['cpu_memory_current']:.1f} MB")
        
        # 처리 시간 통계
        if self.stats['inference_times']:
            inf_times = self.stats['inference_times']
            print(f"\n[Inference Time Per File]")
            print(f"  Mean: {np.mean(inf_times)*1000:.1f} ms")
            print(f"  Median: {np.median(inf_times)*1000:.1f} ms")
            print(f"  Min: {np.min(inf_times)*1000:.1f} ms")
            print(f"  Max: {np.max(inf_times)*1000:.1f} ms")
            print(f"  Std: {np.std(inf_times)*1000:.1f} ms")
        
        if self.stats['preprocessing_times']:
            prep_times = self.stats['preprocessing_times']
            print(f"\n[Preprocessing Time Per File]")
            print(f"  Mean: {np.mean(prep_times)*1000:.1f} ms")
            print(f"  Median: {np.median(prep_times)*1000:.1f} ms")
        
        # 처리량
        if self.stats['inference_times']:
            total_time = sum(self.stats['inference_times'])
            throughput = len(self.stats['inference_times']) / total_time if total_time > 0 else 0
            print(f"\n[Throughput]")
            print(f"  Files/sec: {throughput:.2f}")
            print(f"  Total files: {self.stats['total_files_processed']}")
        
        print("="*60)

try:
    import mediapipe as mp
    MEDIAPIPE_AVAILABLE = True
except ImportError:
    MEDIAPIPE_AVAILABLE = False

# 커스텀 모델 import
sys.path.insert(0, './models')
from vit.stv_transformer_hybrid import vit_base_r50_s16_224_with_recons_iafa

# Simplified MMDet - 학습된 backbone + head만 사용
class MMDet_Simplified(nn.Module):
    """학습된 backbone + head만 사용하는 안전한 구조"""
    def __init__(self, window_size=10):
        super(MMDet_Simplified, self).__init__()
        self.backbone = vit_base_r50_s16_224_with_recons_iafa(
            window_size=window_size, 
            pretrained=False, 
            ckpt_path=None,
            num_classes=0,      # classifier 제거
            global_pool=''      # pooling 제거
        )
        self.head = nn.Linear(768, 2)  # checkpoint에서 로드 가능
                
    def forward(self, x_input):
        x_original, x_recons = x_input
        # Backbone features 추출
        x_st = self.backbone.forward_features(x_input)  # (B, T, 768)
        # Temporal pooling
        x_pooled = torch.mean(x_st, dim=1)              # (B, 768)
        # Classification
        out = self.head(x_pooled)                       # (B, 2)
        return out

### 추론 환경 경로 설정
model_weights_path = "./model/deep-fake-detector-v2-model/MM-Det/current_model.pth"
test_dataset_path = Path("./data")
output_csv_path = Path("submission.csv")

# --- 유틸리티 함수 ---
IMAGE_EXTS = {".jpg", ".jpeg", ".png"}
VIDEO_EXTS = {".avi", ".mp4"}

FACE_DETECTOR = dlib.get_frontal_face_detector()
HAAR_FACE_CASCADE = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')

if MEDIAPIPE_AVAILABLE:
    try:
        mp_face_detection = mp.solutions.face_detection
        MEDIAPIPE_DETECTOR = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.3)
    except:
        MEDIAPIPE_DETECTOR = None
        MEDIAPIPE_AVAILABLE = False
else:
    MEDIAPIPE_DETECTOR = None

def get_boundingbox(face, width, height):
    x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
    size_bb = int(max(x2 - x1, y2 - y1) * 1.3)
    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
    x1 = max(int(center_x - size_bb // 2), 0)
    y1 = max(int(center_y - size_bb // 2), 0)
    size_bb = min(width - x1, size_bb)
    size_bb = min(height - y1, size_bb)
    return x1, y1, size_bb

def detect_face_dlib(image_np, target_size=(224, 224), resize_for_detection=320):
    original_h, original_w = image_np.shape[:2]
    if original_w > resize_for_detection:
        scale = resize_for_detection / float(original_w)
        resized_h = int(original_h * scale)
        resized_np = cv2.resize(image_np, (resize_for_detection, resized_h), interpolation=cv2.INTER_AREA)
    else:
        scale = 1.0
        resized_np = image_np
    faces = FACE_DETECTOR(resized_np, 1)
    if not faces:
        return None
    face = max(faces, key=lambda rect: rect.width() * rect.height())
    scaled_face_rect = dlib.rectangle(
        left=int(face.left() / scale), top=int(face.top() / scale),
        right=int(face.right() / scale), bottom=int(face.bottom() / scale)
    )
    x, y, size = get_boundingbox(scaled_face_rect, original_w, original_h)
    cropped_np = image_np[y:y + size, x:x + size]
    face_img = Image.fromarray(cropped_np).resize(target_size, Image.BICUBIC)
    return face_img

def detect_face_haar(image_np, target_size=(224, 224)):
    gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    faces = HAAR_FACE_CASCADE.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, minSize=(30, 30))
    if len(faces) == 0:
        return None
    x, y, w, h = max(faces, key=lambda f: f[2] * f[3])
    expansion = int(max(w, h) * 0.3)
    x = max(0, x - expansion)
    y = max(0, y - expansion)
    w = min(image_np.shape[1] - x, w + 2 * expansion)
    h = min(image_np.shape[0] - y, h + 2 * expansion)
    cropped_np = image_np[y:y + h, x:x + w]
    face_img = Image.fromarray(cropped_np).resize(target_size, Image.BICUBIC)
    return face_img

def detect_face_mediapipe(image_np, target_size=(224, 224)):
    if MEDIAPIPE_DETECTOR is None:
        return None
    image_rgb = cv2.cvtColor(image_np, cv2.COLOR_RGB2RGB)
    results = MEDIAPIPE_DETECTOR.process(image_rgb)
    if not results.detections:
        return None
    detection = max(results.detections, key=lambda d: d.score[0])
    h, w = image_np.shape[:2]
    bbox = detection.location_data.relative_bounding_box
    x1 = int(bbox.xmin * w)
    y1 = int(bbox.ymin * h)
    x2 = int((bbox.xmin + bbox.width) * w)
    y2 = int((bbox.ymin + bbox.height) * h)
    margin = int(max(x2 - x1, y2 - y1) * 0.1)
    x1 = max(0, x1 - margin)
    y1 = max(0, y1 - margin)
    x2 = min(w, x2 + margin)
    y2 = min(h, y2 + margin)
    cropped_np = image_np[y1:y2, x1:x2]
    if cropped_np.size == 0:
        return None
    face_img = Image.fromarray(cropped_np).resize(target_size, Image.BICUBIC)
    return face_img

def detect_and_crop_face_multi(image: Image.Image, target_size=(224, 224)):
    if image.mode != 'RGB':
        image = image.convert('RGB')
    image_np = np.array(image)
    
    if MEDIAPIPE_AVAILABLE:
        try:
            face_img = detect_face_mediapipe(image_np, target_size)
            if face_img:
                return face_img
        except:
            pass
    
    try:
        face_img = detect_face_dlib(image_np, target_size)
        if face_img:
            return face_img
    except:
        pass
    
    try:
        face_img = detect_face_haar(image_np, target_size)
        if face_img:
            return face_img
    except:
        pass
    
    resized_img = Image.fromarray(image_np).resize(target_size, Image.BICUBIC)
    return resized_img

def process_video_frames(video_path, num_frames=10, max_duration=10):
    face_images = []
    cap = None
    try:
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            return face_images
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total_frames <= 0:
            return face_images
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps > 0:
            max_frames = int(fps * max_duration)
            total_frames = min(total_frames, max_frames)
        frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
        for idx in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if not ret:
                continue
            image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            face_img = detect_and_crop_face_multi(image)
            if face_img:
                face_images.append(face_img)
    except:
        pass
    finally:
        if cap is not None:
            cap.release()
    return face_images

# --- 메인 처리 로직 ---
if __name__ == "__main__" or True:  # Jupyter에서도 작동하도록
    print("="*60)
    print("Starting inference (Simplified MMDet - backbone + head only)...")
    print("="*60)

    # GPU 사용 가능 여부 확인 및 device 설정
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # 리소스 모니터 초기화
    resource_monitor = ResourceMonitor(device=device)
    print(f"✓ Resource monitoring enabled (Device: {device})")

    print(f"\nLoading MMDet_Simplified model...")
    model = MMDet_Simplified(window_size=10)

    # Checkpoint 로드
    checkpoint = torch.load(model_weights_path, map_location='cpu')
    new_state_dict = {}
    for k, v in checkpoint.items():
        if k.startswith('module.'):
            k = k[7:]
        # backbone과 head만 로드 (학습된 가중치만 사용)
        if any(k.startswith(prefix) for prefix in ['backbone', 'head']):
            new_state_dict[k] = v

    load_result = model.load_state_dict(new_state_dict, strict=False)
    print(f"✓ Model loaded: missing={len(load_result.missing_keys)}, unexpected={len(load_result.unexpected_keys)}")

    # 모델을 device로 이동
    model = model.to(device)
    model.eval()
    print(f"✓ MMDet_Simplified ready on {device}")
    
    # 모델 크기 측정
    model_info = resource_monitor.get_model_size(model)
    print(f"\n[Model Information]")
    print(f"  Total Parameters: {model_info['total_params']:,}")
    print(f"  Trainable Parameters: {model_info['trainable_params']:,}")
    print(f"  Model Size: {model_info['size_mb']:.2f} MB")
    
    # 모델 로딩 후 초기 메모리 상태
    resource_monitor.update_stats()
    resource_monitor.print_current_status(prefix="After Model Loading - ")

    # 이미지 전처리
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # 모델 sanity check: 더미 입력으로 추론 테스트
    try:
        with torch.no_grad():
            dummy_input = torch.randn(1, 10, 3, 224, 224).to(device)
            dummy_output = model((dummy_input, dummy_input))
            print(f"✓ Model sanity check passed (output shape: {dummy_output.shape})")
            del dummy_input, dummy_output
            if device == "cuda":
                torch.cuda.empty_cache()
    except Exception as e:
        print(f"✗ Model sanity check FAILED: {type(e).__name__}: {str(e)[:100]}")
        print(f"  This model may not work properly!")

    # CSV 파일 헤더 먼저 작성 (baseline 방식)
    with open(output_csv_path, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "label"])
    print(f"✓ CSV header created")

    # 평가 데이터 파일 목록
    files = [p for p in sorted(test_dataset_path.iterdir()) if p.is_file()]
    total_files = len(files)
    print(f"✓ Test data files: {total_files}")
    print("="*60)

    num_frames_to_extract = 10
    results = []
    error_count = 0
    error_details = []  # 디버깅용 에러 정보 저장
    start_time = time.time()

    # 파일 처리
    for idx, file_path in enumerate(tqdm(files, desc="Processing", ncols=80)):
        face_images = []
        ext = file_path.suffix.lower()
        predicted_class = 0
        
        # 파일별 처리 시간 측정
        file_start_time = time.time()
        prep_start_time = time.time()

        try:
            if ext in IMAGE_EXTS:
                image = Image.open(file_path)
                face_img = detect_and_crop_face_multi(image)
                if face_img:
                    face_images = [face_img] * 10

            elif ext in VIDEO_EXTS:
                face_images = process_video_frames(file_path, num_frames_to_extract, max_duration=10)
                if len(face_images) > 0 and len(face_images) < 10:
                    last_frame = face_images[-1]
                    face_images.extend([last_frame] * (10 - len(face_images)))
                elif len(face_images) == 0:
                    try:
                        cap = cv2.VideoCapture(str(file_path))
                        ret, frame = cap.read()
                        cap.release()
                        if ret:
                            fallback_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                            face_img = detect_and_crop_face_multi(fallback_image)
                            face_images = [face_img] * 10
                    except:
                        pass
            
            # 전처리 시간 기록
            prep_time = time.time() - prep_start_time
            resource_monitor.log_preprocessing_time(prep_time)

            # 추론 수행
            if len(face_images) >= 10:
                inference_start_time = time.time()
                
                with torch.no_grad():
                    batch = face_images[:10]
                    img_tensors = torch.stack([transform(img) for img in batch]).unsqueeze(0).to(device)
                    x_original = img_tensors
                    x_recons = img_tensors
                    
                    logits = model((x_original, x_recons))
                    probs = F.softmax(logits, dim=1)
                    predicted_class = torch.argmax(probs).item()
                    
                    # GPU 메모리 정리
                    del img_tensors, x_original, x_recons, logits, probs
                    if device == "cuda":
                        torch.cuda.empty_cache()
                
                # 추론 시간 기록
                inference_time = time.time() - inference_start_time
                resource_monitor.log_inference_time(inference_time)

        except Exception as e:
            error_count += 1
            # stdout에 출력하지 않고 파일로 저장 (처음 10개만)
            if len(error_details) < 10:
                error_details.append({
                    'filename': file_path.name,
                    'error_type': type(e).__name__,
                    'error_msg': str(e)[:200]  # 메시지 길이 제한
                })
            predicted_class = 0

        # 결과를 리스트에 저장 (baseline 방식)
        results.append([file_path.name, int(predicted_class)])

        # 주기적으로 리소스 상태 업데이트 및 출력
        if (idx + 1) % 500 == 0:
            elapsed = time.time() - start_time
            speed = (idx + 1) / elapsed
            remaining = (total_files - idx - 1) / speed if speed > 0 else 0
            print(f"\n[Progress] {idx+1}/{total_files} | Speed: {speed:.2f} files/s | ETA: {remaining/60:.1f}m")
            
            # 리소스 상태 업데이트 및 출력
            resource_monitor.update_stats()
            resource_monitor.print_current_status(prefix=f"[{idx+1}/{total_files}] ")

    # 최종 통계
    elapsed_total = time.time() - start_time
    print("\n" + "="*60)
    print(f"✓ Inference completed in {elapsed_total/60:.1f} minutes")
    print(f"  Total processed: {len(results)} files")
    print(f"  Speed: {len(results)/elapsed_total:.2f} files/sec")
    print(f"  Errors: {error_count} files")
    print("="*60)

    # CSV에 결과 추가 (append mode, baseline 방식)
    with open(output_csv_path, mode="a", newline="") as f:
        writer = csv.writer(f)
        for row in results:
            writer.writerow(row)

    # CSV 검증 및 상세 분석
    print("\n" + "="*60)
    print("CSV Validation:")
    print(f"  Expected: {total_files} files")
    
    # CSV 파일 읽어서 확인
    with open(output_csv_path, mode="r") as f:
        reader = csv.reader(f)
        rows = list(reader)
        header = rows[0]
        data_rows_list = rows[1:]
        data_rows = len(data_rows_list)
    
    print(f"  Created: {data_rows} rows (excluding header)")
    print(f"  Header: {header}")
    
    if data_rows == total_files:
        print("  ✓ CSV validation PASSED!")
    else:
        print(f"  ✗ FAILED! Missing {total_files - data_rows} rows")
    print(f"  File size: {output_csv_path.stat().st_size:,} bytes")
    
    # 상세 분석
    print("\n" + "="*60)
    print("CSV Content Analysis:")
    print("="*60)
    
    # 1. Label 분포
    label_counts = {}
    for row in data_rows_list:
        if len(row) >= 2:
            label = row[1]
            label_counts[label] = label_counts.get(label, 0) + 1
    print(f"Label distribution: {label_counts}")
    
    # 2. 샘플 데이터 (처음 5개, 마지막 5개)
    print(f"\nFirst 5 rows:")
    for i, row in enumerate(data_rows_list[:5], 1):
        print(f"  [{i}] {row}")
    
    print(f"\nLast 5 rows:")
    for i, row in enumerate(data_rows_list[-5:], data_rows-4):
        print(f"  [{i}] {row}")
    
    # 3. 중복 체크
    filenames_in_csv = [row[0] for row in data_rows_list if len(row) >= 1]
    unique_filenames = set(filenames_in_csv)
    if len(filenames_in_csv) != len(unique_filenames):
        duplicates = len(filenames_in_csv) - len(unique_filenames)
        print(f"\n⚠️  WARNING: {duplicates} duplicate filename(s) found!")
    else:
        print(f"\n✓ No duplicates (all {len(unique_filenames)} filenames are unique)")
    
    # 4. 파일명 매칭 체크
    expected_filenames = set([p.name for p in files])
    csv_filenames = set(filenames_in_csv)
    missing_in_csv = expected_filenames - csv_filenames
    extra_in_csv = csv_filenames - expected_filenames
    
    if missing_in_csv:
        print(f"\n⚠️  Files missing in CSV: {len(missing_in_csv)}")
        for fn in list(missing_in_csv)[:5]:
            print(f"    - {fn}")
        if len(missing_in_csv) > 5:
            print(f"    ... and {len(missing_in_csv)-5} more")
    
    if extra_in_csv:
        print(f"\n⚠️  Extra files in CSV: {len(extra_in_csv)}")
        for fn in list(extra_in_csv)[:5]:
            print(f"    - {fn}")
        if len(extra_in_csv) > 5:
            print(f"    ... and {len(extra_in_csv)-5} more")
    
    if not missing_in_csv and not extra_in_csv:
        print(f"\n✓ All filenames match perfectly!")
    
    # 5. 데이터 타입 체크
    invalid_rows = []
    for i, row in enumerate(data_rows_list, 1):
        if len(row) != 2:
            invalid_rows.append((i, row, f"Wrong column count: {len(row)}"))
        elif row[1] not in ['0', '1']:
            invalid_rows.append((i, row, f"Invalid label: {row[1]}"))
    
    if invalid_rows:
        print(f"\n⚠️  Invalid rows found: {len(invalid_rows)}")
        for idx, row, reason in invalid_rows[:5]:
            print(f"    Row {idx}: {row} - {reason}")
        if len(invalid_rows) > 5:
            print(f"    ... and {len(invalid_rows)-5} more")
    else:
        print(f"\n✓ All rows have valid format (2 columns, labels are 0 or 1)")
    
    print("="*60)
    
    # 추가 검증: 성공한 추론이 있는지 확인
    if error_count == total_files:
        print(f"⚠️  WARNING: ALL files failed! Check details below")
    elif error_count > total_files * 0.5:
        print(f"⚠️  WARNING: High error rate ({error_count}/{total_files})")
    
    print("="*60)
    
    # 디버그 정보 출력 (CSV 검증 후 맨 마지막)
    if error_details:
        print("\n" + "="*60)
        print("DEBUG: Sample Issues (first 10)")
        print("="*60)
        for i, err in enumerate(error_details, 1):
            print(f"\n[{i}] {err['filename']}")
            print(f"    Type: {err['error_type']}")
            print(f"    Info: {err['error_msg']}")
        print("="*60)
    
    # 최종 리소스 사용량 요약
    resource_monitor.update_stats()
    resource_monitor.print_summary()
    
    # 모델 규모 확장 시 예상 리소스 사용량 가이드
    print("\n" + "="*60)
    print("RESOURCE SCALING GUIDE (모델 규모 확장 시 예상치)")
    print("="*60)
    print("\n현재 모델: ViT-Base (MMDet_Simplified)")
    print(f"  - Parameters: {model_info['total_params']:,}")
    print(f"  - Model Size: {model_info['size_mb']:.1f} MB")
    print(f"  - Peak VRAM: {resource_monitor.stats['gpu_memory_peak']:.1f} MB")
    print(f"  - Peak RAM: {resource_monitor.stats['cpu_memory_peak']:.1f} MB")
    
    # 더 큰 모델로 확장 시 예상치 계산
    current_params = model_info['total_params']
    current_vram = resource_monitor.stats['gpu_memory_peak']
    
    print("\n예상 리소스 (모델 크기별):")
    print("┌─────────────────┬──────────────┬───────────────┬──────────────┐")
    print("│ Model           │ Parameters   │ Estimated VRAM│ Batch Size   │")
    print("├─────────────────┼──────────────┼───────────────┼──────────────┤")
    print(f"│ ViT-Base (현재) │ {current_params/1e6:6.1f}M      │ {current_vram:6.1f} MB     │ 1 (현재)     │")
    
    # ViT-Large 예상 (ViT-Base의 약 3.4배 파라미터)
    large_params = current_params * 3.4
    large_vram = current_vram * 2.5  # 메모리는 비선형적으로 증가
    print(f"│ ViT-Large       │ {large_params/1e6:6.1f}M      │ {large_vram:6.1f} MB     │ 1            │")
    
    # Full MM-Det (LLaVA 추가)
    full_vram = current_vram + 10000  # LLaVA 7B 모델 추가 (~10GB)
    print(f"│ Full MM-Det     │ {(current_params + 7e9)/1e6:6.1f}M      │ {full_vram:6.1f} MB     │ 1            │")
    
    # ViT-Huge
    huge_params = current_params * 7.0
    huge_vram = current_vram * 4.0
    print(f"│ ViT-Huge        │ {huge_params/1e6:6.1f}M      │ {huge_vram:6.1f} MB     │ 1            │")
    
    print("└─────────────────┴──────────────┴───────────────┴──────────────┘")
    
    print("\n⚠️  참고사항:")
    print("  - VRAM 추정치는 batch_size=1, fp32 기준입니다")
    print("  - 실제 사용량은 데이터 형태, 최적화 기법에 따라 달라질 수 있습니다")
    print("  - Mixed Precision (FP16) 사용 시 VRAM을 약 40~50% 절감할 수 있습니다")
    print("  - Batch size를 늘리면 VRAM이 비례하여 증가합니다")
    print("="*60)

----

#### 3. `aif.submit()` 함수를 호출하여 최종 결과를 제출

**※주의** : task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 대회 페이지 [베이스라인 코드](https://aifactory.space/task/9197/baseline) 탭에 기재된 가이드라인을 따라 task 별로 확인하실 수 있습니다.
- key가 틀리면 제출이 진행되지 않거나 잘못 제출되므로 task에 맞는 자신의 key를 사용해야 합니다.
-  **NOTE** : 이번 경진대회에서는 3개의 CUDA 버전을 지원하며, 각 CUDA 버전에 따라 task key가 상이합니다. 함수를 실행하기 전에 현재 key가 제출하고자 하는 CUDA 환경에 대한 key인지 반드시 확인하세요.

In [17]:
import aifactory.score as aif
import time
t = time.time()

#-----------------------------------------------------#
aif.submit(model_name="ViT-DeepfakeDetector",
    key="cae0fbcb-0410-4084-a308-21c98d8d886b"
)
#-----------------------------------------------------#
print(time.time() - t)

file : task
jupyter notebook
제출 완료
1657.3978803157806
